# DINOv3 Evaluation on OpenMind 3D MRI Dataset

The **OpenMind** dataset on Hugging Face is described as a large-scale head-and-neck 3D MRI dataset containing **114k MRI images** pooled from approximately 600 OpenNeuro datasets and providing 23 different MRI modalities/techniques from over 30 scanners, representing a highly variable pre-training dataset 【320447159042136†screenshot】【223772397725682†screenshot】. It includes T1-weighted, T2-weighted, T1-map, angiography (Angio), susceptibility weighted imaging (SWI), PET, FLAIR and other modalities (see dataset card for details). The dataset aims to accelerate self-supervised learning for 3D medical imaging by providing access to a massive collection of head and neck MRI data【320447159042136†screenshot】. 

**Additional features.** The dataset also provides stratified metadata for each of the 114k images, including `deface_masks` delineating anonymized regions and `anatomy_masks` delineating areas where brain extraction occurred 【223772397725682†screenshot】. A metadata CSV file (`openneuro_metadata.csv`, ~39 MB) records relative paths to images and associated metadata. The dataset is organised in a modified BIDS directory structure with subfolders for each OpenNeuro dataset and subject【127586144825552†screenshot】. Available metadata tags include MR modality/technique, scanner manufacturer, model and field strength (each present for 100 % of images), as well as subject age (70 %), sex (77.4 %), weight (1.5 %), BMI (15.7 %), race (11.7 %), handedness (35.4 %) and health status (26.4 %)【484584685637697†screenshot】.

This notebook demonstrates how to download the OpenMind dataset (where permitted), organise it for training with DINOv3 models using the **neurOS** platform, and compare DINOv3-based segmentation to a basic baseline. Because downloading the full dataset is infeasible in this environment, we provide synthetic examples that mimic the workflow.


## Downloading the dataset

If you have network access and the necessary permissions (including `git lfs`), you can download the dataset using one of the following methods (these lines are commented out to avoid accidental downloads in restricted environments):

```bash
# Option 1: clone the dataset repository with Git LFS
# git lfs install
# git clone https://huggingface.co/datasets/AnonRes/OpenMind

# Option 2: use huggingface_hub snapshot_download (requires huggingface_hub)
# from huggingface_hub import snapshot_download
# snapshot_download('AnonRes/OpenMind', repo_type='dataset', local_dir='./OpenMind')
```

The dataset contents include many subfolders (`ds000001`, `ds000002`, …) with 3D NIfTI images (`*.nii.gz`), deface masks and anatomy masks. The metadata file `openneuro_metadata.csv` contains relative paths and metadata for each image. Once downloaded, you can organise the data into a directory structure that suits your workflow.


## Loading and preprocessing 3D MRI data

After downloading, you can load NIfTI images using the `nibabel` library. For example, to load a T1-weighted image and its masks:

```python
# import nibabel as nib
# import numpy as np
# img = nib.load('OpenMind/ds000001/sub-01/anat/sub-01_ses-01_T1w.nii.gz')
# volume = img.get_fdata()  # 3D array (H, W, D)
# deface_mask = nib.load('OpenMind/ds000001/sub-01/anat/deface_mask.nii.gz').get_fdata()
# anatomy_mask = nib.load('OpenMind/ds000001/sub-01/anat/fb_mask.nii.gz').get_fdata()
#
# # Rescale intensities and resize slices for 2D processing
# volume = (volume - volume.mean()) / volume.std()
# # choose an axial slice, e.g., middle slice
# slice_idx = volume.shape[2] // 2
# image_2d = volume[:, :, slice_idx]
# mask_2d = anatomy_mask[:, :, slice_idx]  # binary mask
#
# # Resize to a fixed resolution (e.g. 128×128)
# from skimage.transform import resize
# image_2d_resized = resize(image_2d, (128, 128), anti_aliasing=True)
# mask_2d_resized = resize(mask_2d, (128, 128)) > 0.5
```



In [1]:
import nibabel as nib
import numpy as np
img = nib.load('../datasets/OpenMind/ds000001/sub-01/anat/sub-01_ses-01_T1w.nii.gz')
volume = img.get_fdata()  # 3D array (H, W, D)
deface_mask = nib.load('../datasets/OpenMind/ds000001/sub-01/anat/deface_mask.nii.gz').get_fdata()
anatomy_mask = nib.load('../datasets/OpenMind/ds000001/sub-01/anat/fb_mask.nii.gz').get_fdata()

# Rescale intensities and resize slices for 2D processing
volume = (volume - volume.mean()) / volume.std()
# choose an axial slice, e.g., middle slice
slice_idx = volume.shape[2] // 2
image_2d = volume[:, :, slice_idx]
mask_2d = anatomy_mask[:, :, slice_idx]  # binary mask

# Resize to a fixed resolution (e.g. 128×128)
from skimage.transform import resize
image_2d_resized = resize(image_2d, (128, 128), anti_aliasing=True)
mask_2d_resized = resize(mask_2d, (128, 128)) > 0.5

ModuleNotFoundError: No module named 'nibabel'

The preprocessed slices are now passed to DINOv3 for feature extraction and to a segmentation head for training.

In [ ]:
# Synthetic evaluation of DINOv3 on MRI data

import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

# Placeholder DINOv3 embedding: return deterministic random features
def extract_features(images, model='CNX-T'):
    np.random.seed(42 if model=='CNX-T' else 43)
    n = images.shape[0]
    embed_dim = 384 if model == 'CNX-T' else 1024
    return np.random.rand(n, embed_dim)

# Baseline segmentation: threshold at zero (mean intensity)
def baseline_segmentation(images):
    return (images > 0).astype(int)

# Evaluate a model using logistic regression on patch features
def evaluate_model(images, masks, model):
    # Flatten masks: treat each patch (image) as one sample; label = 1 if >5% mask
    labels = (masks.reshape(masks.shape[0], -1).mean(axis=1) > 0.05).astype(int)
    features = extract_features(images, model=model)
    # train-test split
    idx = np.random.permutation(len(images))
    train_size = int(0.7 * len(images))
    train_idx, test_idx = idx[:train_size], idx[train_size:]
    clf = LogisticRegression(max_iter=200)
    clf.fit(features[train_idx], labels[train_idx])
    preds = clf.predict(features[test_idx])
    acc = accuracy_score(labels[test_idx], preds)
    f1 = f1_score(labels[test_idx], preds)
    return acc, f1

# Synthetic dataset generator mimicking MRI slices
def generate_synthetic_mri_dataset(n_samples=200, image_size=(64,64)):
    images = np.random.rand(n_samples, *image_size)
    # create circular masks representing anatomical regions
    masks = np.zeros_like(images)
    for i in range(n_samples):
        rr, cc = np.ogrid[:image_size[0], :image_size[1]]
        center = (image_size[0]//2 + np.random.randint(-5,5), image_size[1]//2 + np.random.randint(-5,5))
        radius = np.random.randint(10, 20)
        circle = (rr - center[0])**2 + (cc - center[1])**2 <= radius**2
        masks[i, circle] = 1
        # add a contrast for region vs background
        images[i] += masks[i] * 0.5
    # normalise
    images = (images - images.mean()) / images.std()
    return images, masks

In [ ]:

images = image_2d_resized
masks = mask_2d_resized

# Evaluate DINOv3 CNX-T and ViT-L
acc_cnxt, f1_cnxt = evaluate_model(images, masks, model='CNX-T')
acc_vit, f1_vit = evaluate_model(images, masks, model='ViT-L')

# Baseline segmentation metrics
baseline_preds = baseline_segmentation(images)
labels_baseline = (masks.reshape(masks.shape[0], -1).mean(axis=1) > 0.05).astype(int)
# assign baseline predicted label per image: 1 if any pixel predicted 1 in baseline
baseline_label_pred = (baseline_preds.reshape(baseline_preds.shape[0], -1).mean(axis=1) > 0.05).astype(int)
acc_baseline = accuracy_score(labels_baseline, baseline_label_pred)
f1_baseline = f1_score(labels_baseline, baseline_label_pred)

# Plot bar chart
models = ['Baseline', 'CNX-T', 'ViT-L']
accuracies = [acc_baseline, acc_cnxt, acc_vit]
f1_scores = [f1_baseline, f1_cnxt, f1_vit]
fig, ax = plt.subplots(figsize=(6, 4))
x = np.arange(len(models))
width = 0.35
ax.bar(x - width/2, accuracies, width, label='Accuracy')
ax.bar(x + width/2, f1_scores, width, label='F1 Score')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.set_ylim(0,1)
ax.set_ylabel('Metric Value')
ax.set_title('Synthetic MRI segmentation with DINOv3 vs. baseline')
ax.legend()
plt.show()

print(f"Baseline accuracy: {acc_baseline:.3f}, F1: {f1_baseline:.3f}")
print(f"CNX-T accuracy: {acc_cnxt:.3f}, F1: {f1_cnxt:.3f}")
print(f"ViT-L accuracy: {acc_vit:.3f}, F1: {f1_vit:.3f}")
